# Laboratorio 6

In [1]:
import json
import pandas as pd
import numpy as np
import warnings
from flare.data_science.features import domain_tld_extract
from urllib.parse import urlparse, urlencode
from flare.data_science.features import dga_classifier
from flare.tools.umbrella import Umbrella
import joblib
import whois
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
def filter_set(dictionary, search_string):
    def iterator_func(x):
        if search_string in list(x.keys()):
            return True
        return False
    return filter(iterator_func, dictionary)

def getDomain(url):
    return urlparse(url).netloc
    
def getProtocol(url):
    return 1 if urlparse(url).scheme == 'https' else 0
    
def getSpecialCharacters(url):
    count_characters = 0
    count_characters += url.count('.')
    count_characters += url.count('-')
    count_characters += url.count('@')
    count_characters += url.count('?')
    count_characters += url.count('&')
    count_characters += url.count('|')
    count_characters += url.count('=')
    count_characters += url.count('_')
    count_characters += url.count('~')
    count_characters += url.count('%')
    count_characters += url.count('/')
    count_characters += url.count('*')
    count_characters += url.count(':')
    count_characters += url.count(',')
    count_characters += url.count(';')
    count_characters += url.count('$')
    count_characters += url.count('%20')
    count_characters += url.count(' ')
    return count_characters

def creation_date(domain):
    return whois.whois(domain).creation_date

### Parte 1

In [3]:
file = open('large_eve.json')
json_file = file.readlines()
file.close()

In [4]:
array = [json.loads(registro) for registro in json_file if registro]

In [5]:
len(array)

746909

In [6]:
dns_records = list(filter_set(array, 'dns'))

In [7]:
for i in range(100, 102):
    print(dns_records[i])

{'timestamp': '2017-07-22T17:33:27.398815-0500', 'flow_id': 2090016796251365, 'pcap_cnt': 54532, 'event_type': 'dns', 'vlan': 150, 'src_ip': '192.168.207.4', 'src_port': 53, 'dest_ip': '192.168.205.170', 'dest_port': 36690, 'proto': 'UDP', 'dns': {'type': 'answer', 'id': 26728, 'rcode': 'NXDOMAIN', 'rrname': '<root>', 'rrtype': 'SOA', 'ttl': 20864}}
{'timestamp': '2017-07-22T17:34:45.072033-0500', 'flow_id': 49722652170593, 'pcap_cnt': 60675, 'event_type': 'dns', 'vlan': 110, 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863', 'src_port': 33150, 'dest_ip': '2001:07fd:0000:0000:0000:0000:0000:0001', 'dest_port': 53, 'proto': 'UDP', 'dns': {'type': 'query', 'id': 31082, 'rrname': 'stork79.dropbox.com', 'rrtype': 'A', 'tx_id': 0}}


In [8]:
df = pd.json_normalize(dns_records)

In [10]:
registros = df[df['dns.rrtype'] == 'A']

In [11]:
registros['dns.rrtype'].value_counts()

A    2849
Name: dns.rrtype, dtype: int64

In [12]:
unique_domains = registros.drop_duplicates(subset = ['dns.rrname'])

In [13]:
unique_domains['tld_extract_domain'] = unique_domains['dns.rrname'].apply(domain_tld_extract)

In [14]:
unique_domains.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_extract_domain
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com


### Parte 2

In [15]:
df_model = pd.DataFrame()
df_model['longitud_hostname'] = unique_domains['dns.rrname'].apply(getDomain).str.len()
df_model['special_characters'] = unique_domains['dns.rrname'].apply(getSpecialCharacters)
df_model['longitud_url'] = unique_domains['dns.rrname'].str.len()
df_model['ratio_digits_url'] = unique_domains['dns.rrname'].str.count('[0-9]') / unique_domains['dns.rrname'].str.len()
df_model['ratio_digits_domain'] = unique_domains['dns.rrname'].apply(getDomain).str.count('[0-9]') / unique_domains['dns.rrname'].apply(getDomain).str.len()

df_model['longitud_hostname'] = df_model['longitud_hostname'].fillna(0)
df_model['special_characters'] = df_model['special_characters'].fillna(0)
df_model['ratio_digits_domain'] = df_model['ratio_digits_domain'].fillna(0)
df_model['longitud_url'] = df_model['longitud_url'].fillna(0)
df_model['ratio_digits_url'] = df_model['ratio_digits_url'].fillna(0)

In [16]:
df_model.head()

,longitud_hostname,special_characters,longitud_url,ratio_digits_url,ratio_digits_domain
0,0,2,20,0.000000,0.0
1,0,2,19,0.105263,0.0
2,0,8,42,0.023810,0.0
6,0,4,36,0.000000,0.0
9,0,2,19,0.000000,0.0


In [17]:
df_model.head()

,longitud_hostname,special_characters,longitud_url,ratio_digits_url,ratio_digits_domain
0,0,2,20,0.000000,0.0
1,0,2,19,0.105263,0.0
2,0,8,42,0.023810,0.0
6,0,4,36,0.000000,0.0
9,0,2,19,0.000000,0.0


In [18]:
modelo = joblib.load('modelo.h5')

In [19]:
predicciones = modelo.predict(df_model)

In [20]:
resultado = []

for i in predicciones:
    if i == 1:
        resultado.append(True)
    else:
        resultado.append(False)

In [21]:
unique_domains[resultado]['tld_extract_domain']

163      192.168.22.110phpmyadmin
1195             "192.168.206.56"
1836              192.168.26-27.0
6898              192.168.21.1201
9979              1922.168.22.254
10048        1922.168.22.254.home
10607             192.168.21-28.0
14730        192.168.21-28.0.home
15507             192.168.22.201:
Name: tld_extract_domain, dtype: object

In [22]:
dga_c = dga_classifier()

[*] Initializing... training classifier - Please wait.
[+] Training classifier on training set
[+] Out of sample legit f1 score 0.9989393832970429
[+] Out of sample dga f1 score 0.6496519721577726
[+] Training final classifier
[+] Classifier Ready


In [23]:
unique_domains['is_legit'] = unique_domains['tld_extract_domain'].apply(dga_c.predict)

In [24]:
unique_domains.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld_extract_domain,is_legit
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com,legit
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com,legit
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net,legit
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home,legit
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com,legit


In [25]:
unique_domains[unique_domains['is_legit'] == 'dga']['tld_extract_domain']

9856     vtlfccmfxlkgifuf.com
10048    1922.168.22.254.home
10646    ejfodfmfxlkgifuf.xyz
14730    192.168.21-28.0.home
Name: tld_extract_domain, dtype: object

### Parte 3

In [26]:
umbrellaCisco = Umbrella(limit = 1000000)

[+] Updated Umbrella Top 1 Million list...


In [27]:
for i in unique_domains[unique_domains['is_legit'] == 'dga']['tld_extract_domain']:
    if umbrellaCisco.domain_tld_in_umbrella(i):
        print(i, ' si se encuentra')
    else:
        print(i, ' no se encuentra')

vtlfccmfxlkgifuf.com  no esta en Cisco Umbrella
1922.168.22.254.home  no esta en Cisco Umbrella
ejfodfmfxlkgifuf.xyz  no esta en Cisco Umbrella
192.168.21-28.0.home  no esta en Cisco Umbrella


In [28]:
for i in unique_domains[unique_domains['is_legit'] == 'dga']['tld_extract_domain']:
    try:
        print(i, ' tiene fecha de creacion ', creation_date(i))
    except:
        print(i, ' no tiene fecha de creacion')

vtlfccmfxlkgifuf.com  no tiene fecha de creacion
1922.168.22.254.home  tiene fecha de creacion de  None
ejfodfmfxlkgifuf.xyz  tiene fecha de creacion de  None
192.168.21-28.0.home  tiene fecha de creacion de  None
